#Задание 1: LDA, NMF

In [1]:
from google.colab import files 
uploaded = files.upload()

Saving habr_texts.txt to habr_texts.txt


In [2]:
!pip install pymorphy2[fast]

     |████████████████████████████████| 51kB 2.8MB/s 
     |████████████████████████████████| 7.1MB 13.6MB/s 
     |████████████████████████████████| 256kB 55.1MB/s 
  Created wheel for DAWG: filename=DAWG-0.7.8-cp36-cp36m-linux_x86_64.whl size=768238 sha256=ba89847d6ea4d9e1abfda139541a416b3214ed72703f8a40a765f898b145a941
  Stored in directory: /root/.cache/pip/wheels/d4/88/d0/4e4abc83eb8f59a71e8dbd8ba99fd5615a3af1fac1ef7f8825
Successfully built DAWG


In [3]:
!pip install pyLDAvis

     |████████████████████████████████| 1.6MB 4.9MB/s 
     |████████████████████████████████| 552kB 57.9MB/s 
  Created wheel for pyLDAvis: filename=pyLDAvis-2.1.2-py2.py3-none-any.whl size=97711 sha256=4ef8147c0e9238135c4469338c558e61e01766a9d8afbc4d7907807dcaae21a3
  Stored in directory: /root/.cache/pip/wheels/98/71/24/513a99e58bb6b8465bae4d2d5e9dba8f0bef8179e3051ac414
  Created wheel for funcy: filename=funcy-1.14-py2.py3-none-any.whl size=32040 sha256=41e9e23f7c0fc84d4ff6a9e79464592952f35cee2c948895983b1ebef3928b3d
  Stored in directory: /root/.cache/pip/wheels/20/5a/d8/1d875df03deae6f178dfdf70238cca33f948ef8a6f5209f2eb
Successfully built pyLDAvis funcy


In [0]:
import gensim
import json
import re
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from pymorphy2 import MorphAnalyzer
import pyLDAvis.gensim
import string
from collections import Counter
import warnings
warnings.filterwarnings("ignore")

morph = MorphAnalyzer()

In [5]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [0]:
stops = set(stopwords.words('russian')) | {'gt',}
def remove_tags(text):
    return re.sub(r'<[^>]+|\d+>', '', text)

def opt_normalize(texts, top=None):
    uniq = Counter()
    for text in texts:
        uniq.update(text)
    
    norm_uniq = {word:morph.parse(word)[0].normal_form for word, _ in uniq.most_common(top)}
    
    norm_texts = []
    for text in texts:
        
        norm_words = [norm_uniq.get(word) for word in text]
        norm_words = [word for word in norm_words if word and word not in stops]
        norm_texts.append(norm_words)
        
    return norm_texts

def tokenize(text):
    words = [word.strip(string.punctuation) for word in text.split()]
    words = [word for word in words if word]
    
    return words

In [0]:
texts = open('habr_texts.txt').read().splitlines()
texts = opt_normalize([tokenize(remove_tags(text.lower())) for text in texts], 30000)

In [0]:

ph = gensim.models.Phrases(texts, scoring='npmi', threshold=0.3) 
p = gensim.models.phrases.Phraser(ph)
ngrammed_texts = p[texts]

In [0]:
dictinary = gensim.corpora.Dictionary(ngrammed_texts)

In [0]:
dictinary.filter_extremes(no_above=0.2, no_below=15)
dictinary.compactify()

In [0]:
corpus = [dictinary.doc2bow(text) for text in ngrammed_texts]

#LDA

1 модель

In [0]:
lda = gensim.models.LdaMulticore(corpus, 200, id2word=dictinary, eval_every=0)

In [13]:
lda.print_topics()

[(156,
  '0.014*"расширение" + 0.011*"сообщение" + 0.006*"компонент" + 0.005*"message" + 0.005*"паттерн" + 0.005*"исследователь" + 0.005*"инициализация" + 0.005*"token" + 0.005*"нативный" + 0.004*"произвольный"'),
 (105,
  '0.020*"игра" + 0.007*"игрок" + 0.004*"исследование" + 0.003*"играть" + 0.003*"адрес" + 0.003*"строка" + 0.003*"прямоугольник" + 0.003*"тест" + 0.003*"текст" + 0.002*"платформа"'),
 (171,
  '0.018*"вакансия" + 0.008*"yahoo" + 0.006*"язык" + 0.005*"опыт_работа" + 0.004*"миллиард" + 0.003*"игра" + 0.003*"выборка" + 0.003*"событие" + 0.003*"знание" + 0.003*"расследование"'),
 (50,
  '0.011*"фсб" + 0.010*"законопроект" + 0.007*"обсуждение" + 0.006*"закон" + 0.005*"сотрудник" + 0.005*"студент" + 0.005*"российский" + 0.004*"опубликовать" + 0.004*"принять" + 0.004*"связать"'),
 (94,
  '0.014*"дефект" + 0.008*"выражение" + 0.005*"if" + 0.004*"пароль" + 0.004*"пакет" + 0.004*"скрипт" + 0.004*"контекст" + 0.004*"return" + 0.003*"данный" + 0.003*"id"'),
 (108,
  '0.021*"электро

Под каждой моделью вынесены более-менее удачные темы, но на них можно не обращать внимание. Сравнение будет в конце блока.


'0.011*"температура" + 0.008*"корпус" + 0.006*"питание" + 0.006*"станция" + 0.005*"резистор" + 0.003*"микроконтроллер" + 0.003*"схема" + 0.003*"домен" + 0.003*"установка" + 0.003*"алгоритм"')

0.012*"подключение" + 0.011*"точка" + 0.009*"публичный" + 0.009*"стиль" + 0.008*"сеть_интернет" + 0.007*"документ" + 0.007*"страница" + 0.007*"внутренний" + 0.005*"dom" + 0.005*"корпоративный_сеть"')]

  '0.011*"фсб" + 0.010*"законопроект" + 0.007*"обсуждение" + 0.006*"закон" + 0.005*"сотрудник" + 0.005*"студент" + 0.005*"российский" + 0.004*"опубликовать" + 0.004*"принять" + 0.004*"связать"')

2 модель

In [0]:
lda = gensim.models.LdaModel(corpus, 200, id2word=dictinary, passes=6, alpha= 'auto')

In [15]:
lda.print_topics()

[(173,
  '0.000*"•" + 0.000*"изображение" + 0.000*"текстура" + 0.000*"сотрудник" + 0.000*"камера" + 0.000*"модуль" + 0.000*"нейросеть" + 0.000*"страна" + 0.000*"внутренний" + 0.000*"точка"'),
 (146,
  '0.166*"решение_уравнение" + 0.000*"директива" + 0.000*"вика" + 0.000*"поток" + 0.000*"массив" + 0.000*"регион" + 0.000*"синхронизация" + 0.000*"итерация_цикл" + 0.000*"алгоритм" + 0.000*"for"'),
 (154,
  '0.407*"communication" + 0.206*"e_e" + 0.000*"1с" + 0.000*"cluster" + 0.000*"узел" + 0.000*"свойство" + 0.000*"фотография" + 0.000*"iot" + 0.000*"автор" + 0.000*"база"'),
 (127,
  '0.303*"зарегистрироваться" + 0.103*"19:00" + 0.098*"познакомить" + 0.074*"принципиально_новый" + 0.018*"гость" + 0.002*"слишком_велик" + 0.000*"соблазн" + 0.000*"очередной" + 0.000*"тестирование" + 0.000*"написание_тест"'),
 (14,
  '0.277*"руководитель_отдел" + 0.150*"деплоить" + 0.134*"деплый" + 0.081*"positive_technologies" + 0.000*"мочь_посоветовать" + 0.000*"байт" + 0.000*"сообщение" + 0.000*"запрос" + 0.0

  '0.296*"valve" + 0.180*"скриншот" + 0.078*"ключ" + 0.078*"крутиться" + 0.065*"каталог" + 0.061*"текстура" + 0.057*"установленный" + 0.031*"включить" + 0.025*"реальный" + 0.013*"адрес"')

  '0.031*"тест" + 0.030*"тестирование" + 0.011*"тестировщик" + 0.009*"проверить" + 0.009*"этап" + 0.009*"тестировать" + 0.009*"фич" + 0.008*"сценарий" + 0.007*"стадия" + 0.007*"кейс"')

  '0.016*"сотрудник" + 0.015*"бизнес" + 0.012*"рынок" + 0.008*"офис" + 0.007*"крупный" + 0.007*"инвестиция" + 0.007*"менеджер" + 0.007*"стартап" + 0.006*"конкурент" + 0.006*"инвестор"'),

'0.013*"анализ" + 0.006*"оценка" + 0.006*"определить" + 0.006*"группа" + 0.005*"исследование" + 0.005*"определять" + 0.005*"требоваться" + 0.004*"определение" + 0.004*"приводить" + 0.004*"фактор"'),

3 модель

In [0]:
lda = gensim.models.LdaModel(corpus, 200, id2word=dictinary, passes=6, alpha= 'asymmetric')

In [17]:
lda.print_topics()

[(199,
  '0.116*"наблюдение" + 0.071*"z" + 0.065*"background" + 0.055*"замедление" + 0.037*"t" + 0.036*"trust" + 0.035*"телескоп" + 0.034*"тест" + 0.029*"tdd" + 0.024*"граф"'),
 (198,
  '0.112*"видео" + 0.048*"регистратор" + 0.032*"стекло" + 0.025*"салон" + 0.023*"бренд" + 0.020*"запись" + 0.020*"камера" + 0.020*"ролик" + 0.019*"gps" + 0.018*"китайский"'),
 (196,
  '0.367*"наушник" + 0.102*"гарнитур" + 0.057*"симулятор" + 0.051*"ухо" + 0.048*"гц" + 0.038*"audio" + 0.035*"пульс" + 0.027*"bluetooth" + 0.023*"динамический" + 0.023*"режим_реальный"'),
 (194,
  '0.168*"шарик" + 0.106*"лампа" + 0.073*"black" + 0.061*"solid" + 0.047*"вход" + 0.046*"лампочка" + 0.035*"высота" + 0.035*"узор" + 0.031*"выход" + 0.030*"выгода"'),
 (197,
  '0.356*"робот" + 0.042*"машина" + 0.037*"мощный" + 0.026*"движение" + 0.023*"яйцо" + 0.023*"робототехника" + 0.022*"огромный" + 0.021*"некий" + 0.016*"платформа" + 0.016*"реальность"'),
 (195,
  '0.163*"опросить" + 0.149*"защититься" + 0.136*"поэтапно" + 0.050*"р

'0.112*"видео" + 0.048*"регистратор" + 0.032*"стекло" + 0.025*"салон" + 0.023*"бренд" + 0.020*"запись" + 0.020*"камера" + 0.020*"ролик" + 0.019*"gps" + 0.018*"китайский"'

  '0.126*"товар" + 0.074*"покупатель" + 0.071*"покупка" + 0.071*"магазин" + 0.066*"заказ" + 0.054*"скидка" + 0.037*"цена" + 0.036*"продажа" + 0.030*"продавец" + 0.025*"предложение"')

'0.016*"сотрудник" + 0.006*"стратегия" + 0.005*"информационный_безопасность" + 0.005*"организация" + 0.005*"технический" + 0.005*"безопасность" + 0.005*"корпоративный_сеть" + 0.005*"администратор" + 0.004*"профиль" + 0.004*"переадресация"'

  '0.028*"иконка" + 0.023*"обновление" + 0.022*"окно" + 0.017*"ui" + 0.014*"jetbrains" + 0.010*"текущий" + 0.009*"ide" + 0.009*"несколько_минута" + 0.009*"навигация" + 0.009*"интерфейс"'

Результаты LDA

Лучше всего, на мой взгяд, справилась третья модель. LdaModel, 6 проходов, alpha= 'asymmetric'. Первая в некоторых случаях выдавала неплохие топики, но остальные были уж слишком размазанными и непонятными. Вторая справлялась уже лучше, неплохих тем было значительно больше, но, в целом, темы недостаточно конкретные. Например, вот: '0.016*"сотрудник" + 0.015*"бизнес" + 0.012*"рынок" + 0.008*"офис" + 0.007*"крупный" + 0.007*"инвестиция" + 0.007*"менеджер" + 0.007*"стартап" + 0.006*"конкурент" + 0.006*"инвестор"

Третья же модель выделяет нормальные темы, достаточно конкретные, а там, где не справляется с конкретикой, хотя бы не слишком сильно залезает в чужие области. Наиболее удачные на мой взгяд: 

Автомобильные аксессуары:
'0.112*"видео" + 0.048*"регистратор" + 0.032*"стекло" + 0.025*"салон" + 0.023*"бренд" + 0.020*"запись" + 0.020*"камера" + 0.020*"ролик" + 0.019*"gps" + 0.018*"китайский"'

(Корпоративная) информационная безопасность:
'0.016*"сотрудник" + 0.006*"стратегия" + 0.005*"информационный_безопасность" + 0.005*"организация" + 0.005*"технический" + 0.005*"безопасность" + 0.005*"корпоративный_сеть" + 0.005*"администратор" + 0.004*"профиль" + 0.004*"переадресация"

Обновления IDE(возможно, конкретно пайчарма):
'0.028*"иконка" + 0.023*"обновление" + 0.022*"окно" + 0.017*"ui" + 0.014*"jetbrains" + 0.010*"текущий" + 0.009*"ide" + 0.009*"несколько_минута" + 0.009*"навигация" + 0.009*"интерфейс"'



1 модель + tfidf

In [0]:
tfidf = gensim.models.TfidfModel(corpus, id2word=dictinary)
corpus = tfidf[corpus]

In [0]:
lda = gensim.models.LdaMulticore(corpus, 200, id2word=dictinary, eval_every=0)

In [20]:
lda.print_topics()

[(16,
  '0.012*"предприятие" + 0.009*"документ" + 0.009*"автоматизировать" + 0.008*"бюджет" + 0.008*"эксплуатация" + 0.007*"заявка" + 0.007*"руководство" + 0.007*"исполнитель" + 0.007*"дисциплина" + 0.006*"согласование"'),
 (57,
  '0.036*"java" + 0.024*"вносить" + 0.020*"конфликт" + 0.016*"вестись" + 0.014*"статус" + 0.012*"локальный_переменный" + 0.012*"прототипирование" + 0.012*"mark" + 0.011*"избежание" + 0.011*"приближение"'),
 (71,
  '0.004*"станция" + 0.003*"webpack" + 0.003*"игра" + 0.003*"иначе" + 0.002*"погода" + 0.002*"билд" + 0.002*"генератор" + 0.002*"смартфон" + 0.002*"for" + 0.002*"linux"'),
 (78,
  '0.003*"скидка" + 0.003*"hpe" + 0.003*"huawei" + 0.003*"ddr4" + 0.002*"бэкап" + 0.002*"видеокарта" + 0.002*"dns" + 0.002*"новогодний" + 0.002*"проложить" + 0.002*"вт"'),
 (11,
  '0.004*"dell" + 0.003*"умный_дом" + 0.003*"билет" + 0.002*"teamcity" + 0.002*"pi" + 0.002*"raspberry_pi" + 0.002*"датчик" + 0.002*"debian" + 0.002*"интервью" + 0.002*"хозяин"'),
 (37,
  '0.009*"статиче

'0.015*"советский" + 0.014*"музыкант" + 0.012*"фильтр" + 0.010*"министерство" + 0.009*"усилитель" + 0.007*"звучание" + 0.007*"создатель" + 0.007*"звук" + 0.006*"производство" + 0.006*"искажение"

'0.017*"браузер" + 0.017*"messages" + 0.012*"edge" + 0.011*"lt;?php" + 0.010*"microsoft" + 0.009*"return" + 0.008*"умолчание" + 0.008*"windows_10" + 0.007*"логин" + 0.007*"двигать"

2 модель + tfidif

In [0]:
lda = gensim.models.LdaModel(corpus, 200, id2word=dictinary, passes=6, alpha= 'auto')

In [22]:
lda.print_topics()

[(174,
  '0.014*"devops" + 0.000*"scala" + 0.000*"вакансия" + 0.000*"progress" + 0.000*"региональный" + 0.000*"зарплата" + 0.000*"набирать_оборот" + 0.000*"разрыв" + 0.000*"вырасти" + 0.000*"java"'),
 (54,
  '0.000*"15_000" + 0.000*"потомок" + 0.000*"storage" + 0.000*"вм" + 0.000*"защита" + 0.000*"удаление" + 0.000*"уничтожение" + 0.000*"акция" + 0.000*"узел" + 0.000*"диск"'),
 (87,
  '0.021*"office_365" + 0.001*"office" + 0.000*"фигура" + 0.000*"составной" + 0.000*"microsoft_office" + 0.000*"ход" + 0.000*"доска" + 0.000*"аргумент_польза" + 0.000*"running" + 0.000*"последний_версия"'),
 (199,
  '0.011*"раствор" + 0.007*"медь" + 0.000*"вольт" + 0.000*"цивилизация" + 0.000*"ui" + 0.000*"поддаваться" + 0.000*"учитель" + 0.000*"ux" + 0.000*"отдел" + 0.000*"личный_кабинет"'),
 (166,
  '0.010*"просить_кат" + 0.000*"gdb" + 0.000*"старение" + 0.000*"сан-франциско" + 0.000*"резервный_копирование" + 0.000*"авторизоваться" + 0.000*"void_void" + 0.000*"указываться" + 0.000*"убить" + 0.000*"monitor

'0.108*"аккумулятор" + 0.096*"батарея" + 0.050*"ячейка" + 0.035*"производство" + 0.014*"вверху" + 0.010*"внизу" + 0.010*"рекурсия" + 0.007*"тонкий" + 0.005*"корпорация" + 0.004*"нагреваться"

'0.013*"атрибут" + 0.010*"страница" + 0.009*"state" + 0.009*"модуль" + 0.008*"браузер" + 0.008*"узел" + 0.008*"пиксель" + 0.008*"индекс" + 0.008*"html" + 0.008*"api"

'0.007*"запрос" + 0.006*"поток" + 0.005*"пакет" + 0.005*"интерфейс" + 0.005*"трафик" + 0.005*"база" + 0.005*"проверка" + 0.005*"строка" + 0.004*"скрипт" + 0.004*"услуга"

3 модель + tfidf

In [0]:
lda = gensim.models.LdaModel(corpus, 200, id2word=dictinary, passes=6, alpha= 'asymmetric')

In [25]:
lda.print_topics()

[(198,
  '0.000*"ученик" + 0.000*"индивидуальный_подход" + 0.000*"watson" + 0.000*"преподаватель" + 0.000*"ibm" + 0.000*"учитель" + 0.000*"школьный" + 0.000*"образование" + 0.000*"школа" + 0.000*"школьник"'),
 (196,
  '0.000*"прелесть" + 0.000*"разблокировать" + 0.000*"альтернативный_вариант" + 0.000*"взломать" + 0.000*"время_сутки" + 0.000*"денежный" + 0.000*"детализация" + 0.000*"окей" + 0.000*"переадресация" + 0.000*"gmail"'),
 (199,
  '0.000*"станция" + 0.000*"ux" + 0.000*"ui" + 0.000*"космический" + 0.000*"звонок" + 0.000*"марс" + 0.000*"диаметр" + 0.000*"видео" + 0.000*"взаимодействие" + 0.000*"километр"'),
 (191,
  '0.009*"деплоить" + 0.000*"деплый" + 0.000*"iis" + 0.000*"дизайнер" + 0.000*"релиз" + 0.000*"достижение_цель" + 0.000*"книга" + 0.000*"портфолио" + 0.000*"хост" + 0.000*"сущность"'),
 (194,
  '0.000*"ассет" + 0.000*"заголовок" + 0.000*"запрос" + 0.000*"игра" + 0.000*"префикс" + 0.000*"u" + 0.000*"подстановка" + 0.000*"модификатор" + 0.000*"документ" + 0.000*"игрок"'),

'0.000*"ученик" + 0.000*"индивидуальный_подход" + 0.000*"watson" + 0.000*"преподаватель" + 0.000*"ibm" + 0.000*"учитель" + 0.000*"школьный" + 0.000*"образование" + 0.000*"школа" + 0.000*"школьник"

'0.012*"антивирус" + 0.000*"антивирусный" + 0.000*"специалист_безопасность" + 0.000*"язык" + 0.000*"google" + 0.000*"открытый_ключ" + 0.000*"аудит" + 0.000*"конференция" + 0.000*"информационный_безопасность" + 0.000*"компьютер"

'0.057*"галактика" + 0.043*"i=0_i" + 0.019*"материя" + 0.014*"тёмный_материя" + 0.011*"i" + 0.011*"гравитация" + 0.010*"исходный_положение" + 0.008*"cortana" + 0.000*"std::cout_lt;&lt" + 0.000*"ревить"

Результаты LDA + tfidf

C tfidf результаты ухудшились, однако интересно то, что с таким подходом хотя бы видна какая-то логика размазывания. Темы, даже не самые хорошие, всё равно более точны (темы с играми обросли словами типа 'конференция', 'доклад', а значит в текстах речь, скорее всего, шла о конференции разработчиков игр). Третья модель на этот раз тоже выделила вполне нормальные темы, но сильнее размазалась в и без того не самых удачных вариантах. Тем не менее, другие модели показали результаты ещё хуже. Список наиоболее удачных тем для третьей модели: 

Образование:
'0.000*"ученик" + 0.000*"индивидуальный_подход" + 0.000*"watson" + 0.000*"преподаватель" + 0.000*"ibm" + 0.000*"учитель" + 0.000*"школьный" + 0.000*"образование" + 0.000*"школа" + 0.000*"школьник"

Информационная безопасность:
'0.012*"антивирус" + 0.000*"антивирусный" + 0.000*"специалист_безопасность" + 0.000*"язык" + 0.000*"google" + 0.000*"открытый_ключ" + 0.000*"аудит" + 0.000*"конференция" + 0.000*"информационный_безопасность" + 0.000*"компьютер"

Космос, физика (ещё и Halo приплела):
'0.057*"галактика" + 0.043*"i=0_i" + 0.019*"материя" + 0.014*"тёмный_материя" + 0.011*"i" + 0.011*"гравитация" + 0.010*"исходный_положение" + 0.008*"cortana" + 0.000*"std::cout_lt;&lt" + 0.000*"ревить"

#NMF

In [0]:
from sklearn.decomposition import NMF
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import pandas as pd

In [0]:
stexts = [' '.join(text) for text in texts]

1 модель

In [0]:
vectorizer = TfidfVectorizer(max_features=750, min_df=6, max_df=0.2, ngram_range=(1,3))
X = vectorizer.fit_transform(stexts)

In [0]:
model = NMF(n_components=30)

In [31]:
model.fit(X)

NMF(alpha=0.0, beta_loss='frobenius', init=None, l1_ratio=0.0, max_iter=200,
    n_components=30, random_state=None, shuffle=False, solver='cd', tol=0.0001,
    verbose=0)

In [0]:
feat_names = vectorizer.get_feature_names()

In [0]:
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

In [36]:
top_words = model.components_.argsort()[:,:-5:-1]

for i in range(top_words.shape[0]):
    words = [feat_names[j] for j in top_words[i]]
    print(i, "  ".join(words))

0 сотрудник  бизнес  заказчик  деньга
1 public  new  void  класс
2 игра  игрок  игровой  играть
3 космический  аппарат  земля  станция
4 процессор  память  intel  диск
5 the  to  of  and
6 рынок  российский  страна  миллион
7 lt  gt  div  lt div
8 смартфон  мобильный  телефон  android
9 адрес  ip  ключ  пакет
10 google  android  реклама  api
11 доклад  конференция  участник  видео
12 атака  безопасность  уязвимость  защита
13 обучение  алгоритм  программирование  изображение
14 сигнал  частота  канал  диапазон
15 камера  видео  изображение  кадр
16 дата центр  центр  дата  оборудование
17 учёный  мозг  исследование  университет
18 тест  тестирование  end  тестовый
19 windows  microsoft  linux  studio
20 if  int  return  amp
21 печать  3d  материал  производство
22 javascript  страница  браузер  react
23 бот  сообщение  канал  робот
24 запрос  таблица  строка  запись
25 виртуальный  машина  автомобиль  реальность
26 книга  читать  часы  профессиональный
27 вселенная  энергия  свет  теор

In [37]:
model.reconstruction_err_

54.85402949571099

2 игра  игрок  игровой  играть

3 космический  аппарат  земля  станция

4 процессор  память  intel  диск

8 смартфон  мобильный  телефон  android

2 модель 

In [0]:
vectorizer = TfidfVectorizer(max_features=1000, min_df=4, max_df=0.2, ngram_range=(1,3))
X = vectorizer.fit_transform(stexts)

In [0]:
model = NMF(n_components=40)

In [47]:
model.fit(X)

NMF(alpha=0.0, beta_loss='frobenius', init=None, l1_ratio=0.0, max_iter=200,
    n_components=40, random_state=None, shuffle=False, solver='cd', tol=0.0001,
    verbose=0)

In [0]:
feat_names = vectorizer.get_feature_names()

In [49]:
top_words = model.components_.argsort()[:,:-5:-1]

for i in range(top_words.shape[0]):
    words = [feat_names[j] for j in top_words[i]]
    print(i, "  ".join(words))

0 программист  курс  кто  программирование
1 int  amp  lt  char
2 игра  игрок  игровой  играть
3 учёный  клетка  исследование  исследователь
4 процессор  память  intel  ядро
5 lt  gt  div  lt div
6 рынок  российский  страна  миллион
7 public  void  string  new
8 the  to  of  and
9 страница  браузер  реклама  контент
10 windows  microsoft  linux  ос
11 доклад  конференция  участник  видео
12 атака  уязвимость  безопасность  защита
13 космический  спутник  аппарат  земля
14 дата центр  центр  дата  оборудование
15 печать  принтер  3d  материал
16 мозг  пациент  глаз  движение
17 обучение  алгоритм  ия  изображение
18 тест  тестирование  тестовый  test
19 модуль  строка  скрипт  плагин
20 виртуальный  машина  виртуальный машина  резервный
21 бот  сообщение  канал  api
22 var  function  return  new
23 автомобиль  машина  карта  метр
24 запрос  таблица  запись  база дать
25 вселенная  энергия  теория  свет
26 книга  часы  читать  профессиональный
27 google  android  мобильный  ios
28 товар 

In [50]:
model.reconstruction_err_

54.40648298890789

3 модель 

In [0]:
vectorizer = CountVectorizer(max_features=500, min_df=12, max_df=0.2, ngram_range=(1,3))
X = vectorizer.fit_transform(stexts)

In [0]:
model = NMF(n_components=30)

In [58]:
model.fit(X)

NMF(alpha=0.0, beta_loss='frobenius', init=None, l1_ratio=0.0, max_iter=200,
    n_components=30, random_state=None, shuffle=False, solver='cd', tol=0.0001,
    verbose=0)

In [0]:
feat_names = vectorizer.get_feature_names()

In [60]:
top_words = model.components_.argsort()[:,:-5:-1]

for i in range(top_words.shape[0]):
    words = [feat_names[j] for j in top_words[i]]
    print(i, "  ".join(words))

0 up  11  23  50
1 gt  lt  gt lt  function
2 sec  ms  тест  пакет
3 if  else  return  string
4 the  to  is  in
5 переменный  поль  канал  кнопка
6 рынок  сотрудник  бизнес  деньга
7 игра  игрок  игровой  играть
8 public  new  void  string
9 end  then  if  not
10 set  as  on  null
11 адрес  пакет  ip  трафик
12 result  function  for  new
13 int  for  блок  массив
14 user  do  end  тест
15 кнопка  char  иначе  символ
16 страница  карта  строка  скрипт
17 сетевой  пакет  протокол  обработка
18 type  data  name  string
19 класс  библиотека  строка  блок
20 копия  объём  канал  запись
21 запрос  from  таблица  запись
22 центр  дата  услуга  инфраструктура
23 of  and  for  научный
24 amp  amp amp  null  char
25 виртуальный  реальность  машина  windows
26 return  if  function  var
27 память  ядро  процессор  производительность
28 сигнал  модуль  камера  частота
29 lt  lt lt  gt  return


Резлуьтаты NMF

Наиболее успешно справилась вторая модель (Tfidf векторизатор, 1000 фичей, min_df=4, max_df=0.2, 40 тем). От первой отличается, в основном, количеством тем. Третья модель построенная на countvectorizer выдала совсем плохие результаты.

Модели NMF в данном случае справились намого лучше, чем LDA. Сложно выделить три наилучших темы, потому что процентов 60 топа - очень даже ничего. 

2 игра  игрок  игровой  играть

3 учёный  клетка  исследование  исследователь

4 процессор  память  intel  ядро

12 атака  уязвимость  безопасность  защита

13 космический  спутник  аппарат  земля

14 дата центр  центр  дата  оборудование

15 печать  принтер  3d  материал

Вот такие, например. 

